In [1]:
# TEMPLATE: https://github.com/OpenSourceBrain/CalciumImagingDriftingGrating/blob/master/Convert.py
# SAMPLE NOTEBOOK: https://github.com/NeurodataWithoutBorders/nwb_tutorial/blob/master/HCK08/ophys_tutorial.ipynb

import glob, os
from datetime import datetime
from dateutil.tz import tzlocal
import numpy as np
import pynwb
from pynwb.misc import AnnotationSeries
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, \
    Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries


#REQUIRED: session_id, experiment_description, session_start_time
# References after fields refer to DB (active_atlas_production)

#PULL FROM DB
description = 'Description	Wide-field imaging of concurrent pan-neuronal activity and pial arterial smooth muscle activity across the cortical mantle during resting, stimulus, and after-stimulus time periods' #description
identifier = 'Wide-field imaging of concurrent pan-neuronal activity and pial arterial smooth muscle activity across the cortical mantle during resting, stimulus, and after-stimulus time periods.' #identifier
session_start_time = datetime.utcnow() #session_start_time
session_id = '08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hz' #session_id
experimenter = 'Jacob Duckworth'  #FK_annotator_id -> name(s)
institution='UCSD' #FK_performance_lab_id -> FK_performance_center_id -> name
doi_pub='' #doi_pub
lab_performance_center='David Kleinfeld' #FK_performance_lab_id -> name

nwbfile = NWBFile(
    identifier=identifier,
    session_description=description,
    session_start_time=session_start_time,
    session_id=session_id,
    experimenter=experimenter,
    institution=institution,
    related_publications=doi_pub,
    lab=lab_performance_center,
)
print(nwbfile)
print(nwbfile.object_id)

#PUSH TO DB
objectid_uuid = nwbfile.object_id

root pynwb.file.NWBFile at 0x140630432307280
Fields:
  experimenter: ['Jacob Duckworth']
  file_create_date: [datetime.datetime(2022, 9, 6, 11, 23, 2, 625641, tzinfo=tzlocal())]
  identifier: Wide-field imaging of concurrent pan-neuronal activity and pial arterial smooth muscle activity across the cortical mantle during resting, stimulus, and after-stimulus time periods.
  institution: UCSD
  lab: David Kleinfeld
  related_publications: ['']
  session_description: Description	Wide-field imaging of concurrent pan-neuronal activity and pial arterial smooth muscle activity across the cortical mantle during resting, stimulus, and after-stimulus time periods
  session_id: 08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hz
  session_start_time: 2022-09-06 18:23:02.624568-07:00
  timestamps_reference_time: 2022-09-06 18:23:02.624568-07:00

27320773-8d59-4336-b215-752305495469


/opt/pipeline38/lib/python3.8/site-packages/pynwb/file.py:1100: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [17]:
#EXPERIMENTAL SUBJECT - OPTIONAL
from pynwb.file import Subject

species = 'Mus musculus - NCBI:txid10090' #species -> latin  + NCBI (MOUSE = 'Mus musculus - NCBI:txid10090')
exp_description = '' #animal.comments
subject_id = 'JD220307M3' #animal.id
age = 'P93D' #experiment_date - animal.date_of_birth? ;ref: #https://en.wikipedia.org/wiki/ISO_8601#Durations
date_of_birth = ''
sex = 'M' #animal.sex
genotype = 'B6.Cg-Tg(Acta2-GCaMP8.1/mVermilion)B34-4Mik/J x Tg(Thy1-jRGECO1a)GP8.20Dkim/J' #animal.genotype (mother/father)

nwbfile.subject = Subject(
    subject_id=subject_id,
    age=age,
    description=exp_description,
    species=species,
    genotype=genotype,
    sex=sex
)

In [18]:
#BEHAVIOR & PHYSIOLOGICAL MEASUREMENTS (surgery) - OPTIONAL
from pynwb.file import NWBFile

surgery = '2 hemisphere thin-skull window'
pharmacology = ''
PupilCamera = True
BodyCamera = True
Accelerometer_description = 'Yes, in hammock under animal'
experiment_description = f'PupilCamera = {PupilCamera}; BodyCamera = {BodyCamera}; Accelerometer_description: {Accelerometer_description}'

stimulus_notes = 'Left eye light pulses at 0.14 Hz, 2 second pulse length, 10ms on/off bursts'

nwbfile.file = NWBFile(
    surgery=surgery,
    pharmacology=pharmacology,
    identifier=identifier,
    session_description=description,
    session_start_time=session_start_time,
    session_id=session_id,
    experimenter=experimenter,
    institution=institution,
    related_publications=doi_pub,
    lab=lab_performance_center,
    experiment_description=experiment_description,
    stimulus_notes=stimulus_notes
)


In [19]:
#DATA ACQUISITION - OPTIONAL
#ref: https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html

device_name = 'Wide-field Microscope' #
device_description1 = "Zeiss SteREO Discovery.V8"
emission_filter = "Chroma ZET 488/561m" #part of 'device_description'
dichoroic_mirror = "Chroma 470dcxr" #part of 'device_description'
camera = "Photometrics Prime 95B" #part of 'device_description'
camera_magnification = "1.25x" #part of 'device_description'
camera_exposure_time = "100 ms" #part of 'device_description'
device_mfg = "Zeiss"
optical_ch_name1 = ""
optical_ch_desc1 = "Chroma ET470/40x" #excitation filter
optical_ch_name2 = "Semrock FF01-560/14" #excitation filter
optical_ch_desc2 = ""
imaging_plane_indicator1 = "GCaMP"
imaging_plane_indicator2 = "jRGECO"
optical_emission_lambda1 = 515 #wavelength in nm; float
optical_emission_lambda2 = 590 #wavelength in nm; float
imaging_plane_name1 = "Thorlabs M470L4"
imaging_plane_name2 = "Thorlabs M565L3 LED"
imaging_plane_rate1 = 5 #rate images are acquired, in Hz; float
imaging_plane_rate2 = 5 #rate images are acquired, in Hz; float
imaging_plane_description1 = "alternating excitation"
imaging_plane_description2 = "alternating excitation"
imaging_plane_excitation_lambda1 = 470 #wavelength in nm; float
imaging_plane_excitation_lambda2 = 565 #wavelength in nm; float

#COMPOSITE DESCRIPTION
device_description = f'device_description: {device_description1}; emission_filter: {emission_filter}; dichoroic_mirror: {dichoroic_mirror}; camera: {camera}; camera_magnification: {camera_magnification}; camera_exposure_time: {camera_exposure_time}'

device = nwbfile.create_device(
    name=device_name,
    description=device_description,
    manufacturer=device_mfg
)
optical_channel1 = OpticalChannel(
    name=optical_ch_name1,
    description=optical_ch_desc1,
    emission_lambda=float(optical_emission_lambda1)
)
optical_channel2 = OpticalChannel(
    name=optical_ch_name1,
    description=optical_ch_desc1,
    emission_lambda=float(optical_emission_lambda1)
)

#need input from Jacob
imaging_plane_indicator = imaging_plane_location = ''
image_series1_name = 'NULL NAME 1'
image_series2_name = 'NULL NAME 2'
image_series1_rate = float(imaging_plane_rate1)

image_series1_external_file = [
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hz_Parameters.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzbeforepuff_neu_intp.fig',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzbeforepuff_ves_intp.fig',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzpuff_neu_intp.fig',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzpuff_ves_intp.fig',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_neurons_wave_intp.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_neurons_wave.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_neurons.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_toplot_intp.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_wave_intp.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff_wave.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_beforepuff.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_neurons_wave_intp.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_neurons_wave.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_neurons.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_toplot_intp.mat',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_wave_intp.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff_wave.h5',
'08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hzsvd_puff.mat',
'Mask.tif',
'Mask1.tif',
'rim.mat'
]

image_series2_external_file = ['Mask.tif',
'Mask1.tif',
'rim.mat'
] #must be list of files

# imaging_plane_indicator = "GFP"
# imaging_plane_location = "V1"
# imaging_plane_grid_spacing = [.01, .01]
# imaging_plane_grid_spacing_unit = "meters"
# imaging_plane_origin_coords = [1., 2., 3.]
# imaging_plane_origin_coords_unit = "meters"
# image_series1_name = 'TwoPhotonSeries1'
# image_series1_dimension = [100, 100]
# image_series1_external_file = ['images.tiff']
# image_series1_starting_frame = [0]
# image_series1_starting_time = 0.0
# image_series1_rate = 1.0



imaging_plane1 = nwbfile.create_imaging_plane(
    name=imaging_plane_name1,
    optical_channel=optical_channel1,
    imaging_rate=float(imaging_plane_rate1),
    description=imaging_plane_description1,
    device=device,
    excitation_lambda=float(imaging_plane_excitation_lambda1),
    indicator=imaging_plane_indicator1,
    location=imaging_plane_location
    # grid_spacing=imaging_plane_grid_spacing,
    # grid_spacing_unit=imaging_plane_grid_spacing_unit,
    # origin_coords=imaging_plane_origin_coords,
    # origin_coords_unit=imaging_plane_origin_coords_unit
)
image_series1 = TwoPhotonSeries(
    name=image_series1_name,
    # dimension=image_series1_dimension,
    external_file=image_series1_external_file,
    imaging_plane=imaging_plane1,
    # starting_frame=image_series1_starting_frame,
    format='external',
    # starting_time=image_series1_starting_time,
    rate=image_series1_rate
)
imaging_plane2 = nwbfile.create_imaging_plane(
    name=imaging_plane_name2,
    optical_channel=optical_channel2,
    imaging_rate=float(imaging_plane_rate2),
    description=imaging_plane_description2,
    device=device,
    excitation_lambda=float(imaging_plane_excitation_lambda2),
    indicator=imaging_plane_indicator2,
    location=imaging_plane_location,
    # grid_spacing=imaging_plane_grid_spacing,
    # grid_spacing_unit=imaging_plane_grid_spacing_unit,
    # origin_coords=imaging_plane_origin_coords,
    # origin_coords_unit=imaging_plane_origin_coords_unit
)
image_series2 = TwoPhotonSeries(
    name=image_series2_name,
    # dimension=image_series1_dimension,
    external_file=image_series2_external_file,
    imaging_plane=imaging_plane2,
    # starting_frame=image_series1_starting_frame,
    format='external',
    # starting_time=image_series1_starting_time,
    rate=image_series1_rate
)
nwbfile.add_acquisition(image_series1)
nwbfile.add_acquisition(image_series2)

In [20]:
#EXPERIMENTAL DESIGN - OPTIONAL
animal_position_description = 'Head fixed'
stimuli_description = 'Left eye light pulses at 0.14 Hz, 2 second pulse length, 10ms on/off bursts'
trial_structure_description = 'Before-stim = 250 s  Stim = 500 s  After-stim = 250 s'
nwbfile.add_trial_column(name='animal_position', description=animal_position_description)
nwbfile.add_trial_column(name='stimuli', description=stimuli_description)
nwbfile.add_trial_column(name='trial_structure', description=trial_structure_description)

In [31]:
#NEURONAL ACTIVITY - OPTIONAL

activity_measurements_modality = 'optical'
activity_measurement_description = 'L2/3, L5 neuronal activity reported optically by the genetically encoded jRGECO calcium indicator under the Thy1 promoter'

pynwb.misc.AnnotationSeries.name = f'activity_measurements_modality: {activity_measurements_modality}'
pynwb.misc.AnnotationSeries.description = f'activity_measurement_description: {activity_measurement_description}'

In [127]:
#DATA ACQUISITION - OPTIONAL
ext_files_path = u"/mnt/e/temp/rui/"

image_series_external_files = {}
for root, dirs, files in os.walk(ext_files_path):
    path = root.split(os.sep)
    folder_name = os.path.basename(root)

    image_series = []
    for file in files:
        file_name = file
        c_time = os.path.getctime(os.path.join(ext_files_path, folder_name, file))
        dt_creation_time = datetime.fromtimestamp(c_time)
        creation_time = datetime.fromtimestamp(c_time)
        #creation_time = dt_creation_time.strftime("%Y-%b-%d %H:%M:%S")
        image_series.append([os.path.join(folder_name, file_name), creation_time])
    if len(folder_name) > 0:
        image_series_external_files[folder_name] = image_series

image_series = []
for key, files in image_series_external_files.items():
    description = f'{key}'
    timestamp = files[1]
    comments = f'FILES {len(files)}'

    ext_files = []
    ext_files_timestamps = []
    for filename in files:
        ext_files.append(files[0][0])
        #ext_files_timestamps.append(files[0][1])
        dt = datetime(2018, 10, 22, 0, 0)

        timestamp = 1563907835.857213
        ext_files_timestamps = np.arange(len(files)) + timestamp



    image_series.append(ImageSeries(name=f'image_series_{description}',
                                   external_file=ext_files,
                                   starting_frame=[0],
                                   timestamps=ext_files_timestamps,
                                   format='external',
                                   description=description,
                                   comments=comments))
for series in image_series:
    nwbfile.add_acquisition(series)

image_series

In [62]:
#OUTPUT OF NWB FILE (COULD ALSO GO TO DATABASE)
nwb_path = f"/mnt/e/temp/{session_id}.nwb"


nwb_file_name = nwb_path
with NWBHDF5IO(nwb_file_name, 'w') as io:
    io.write(nwbfile)

print(f"Written NWB file to {nwb_file_name}")

Written NWB file to /mnt/e/temp/08.Jun.2022_19.19.34_JD220307M3_PwrG80R80_leftphot_0.14Hz.nwb


In [ ]:
#VALIDATE NWB FILE

#python -m nwb.validate filename.nwb